In [1]:
import os
import pandas as pd
import pickle
from library.scoreCalculatorLibrary import calculate_score

In [2]:
input_dir = r"../data/54ft/"
output_dir=r"../reports/54ft/"

#file Name
zone_names=list(map(lambda e:f"{e}_test_split.csv",["station","dvc_more","54ft_road","junction_mall","prantika_bus_stand"]))

_7day_test=["54ft_test.csv"]

In [3]:
#Helper functions
def remove_with_respect_to_poi_df(df,bus=0,turn=0,signal=0,congestion=0,adhoc=0):
    return df[~((df['Prediction Is_Bus_stop']==bus)&(df['Prediction Is_Turn']==turn)&(df['Prediction Is_Signal']==signal)&(df['Prediction Is_Congestion']==congestion)&(df['Prediction Is_Adhoc']==adhoc))]

# converts hour:minute:seconds to seconds
def time_as_int(timestr):
    hh, mm, ss = timestr.split(":")
    time_int = (int(hh)*3600)+(int(mm)*60)+int(ss)
    return time_int

# Scoring

In [4]:
overall=[] #overall array

for test_df_name in (zone_names+_7day_test):#iterator name

    zoneName=test_df_name.split('_test_split.csv')[0].split('.')[0]
    input_filename = os.path.join(input_dir,f"Predictions_merged_54ft_{zoneName}_zone.csv")

    os.makedirs(output_dir + zoneName, exist_ok=True) #outer directory
    os.makedirs(output_dir + zoneName + "/DateWise", exist_ok=True) #date inner scores


    #Getting the data for Zone to score
    df=pd.read_csv(input_filename)
    df=remove_with_respect_to_poi_df(df,bus=0,turn=1,signal=0,congestion=0,adhoc=0) #filter only turn
    df=remove_with_respect_to_poi_df(df,bus=0,turn=0,signal=0,congestion=0,adhoc=0) #filter no poi


    df['start_time']=df["instance_start_time"].copy() #copying start_time
    df['end_time']=df["instance_end_time"].copy() #copying end_time

    df["instance_start_time"] = df["instance_start_time"].apply(lambda x: time_as_int(x)) #convert to sec
    df["instance_end_time"] = df["instance_end_time"].apply(lambda x: time_as_int(x)) #convert to sec


    date_groups=df.groupby('instance_date') #date wise groups

    for date in date_groups.groups.keys(): #loop over dates
        date_data=date_groups.get_group(date) #get date data
        overall_score, list_scores, no_of_instances = calculate_score(date_data) #cal score
        date_data['scores']=list_scores #record scores

        d=date.split("/")                   #date formating
        d='-'.join([d[e] for e in [2,0,1]]) #date formating

        #saving scores for all instances of a perticular day
        date_data[['start_time','end_time','scores']].to_csv(output_dir + zoneName + "/DateWise"+f"/{d}.csv",index=False)

        print(f"ON ZONE: {zoneName} ON DATE:{d}")

        #appending to overall array "date scores"
        overall.append({'zone':zoneName,'date':d,'overall_score':overall_score})

#saving overall scores over days
pd.DataFrame(overall).to_csv(output_dir +"/overall.csv",index=False)


ON ZONE: station ON DATE:2019-06-28
ON ZONE: station ON DATE:2019-06-30
ON ZONE: station ON DATE:2019-07-01
ON ZONE: station ON DATE:2019-07-02
ON ZONE: station ON DATE:2019-07-03
ON ZONE: station ON DATE:2019-07-04
ON ZONE: station ON DATE:2019-07-05
ON ZONE: dvc_more ON DATE:2019-06-28
ON ZONE: dvc_more ON DATE:2019-06-30
ON ZONE: dvc_more ON DATE:2019-07-01
ON ZONE: dvc_more ON DATE:2019-07-02
ON ZONE: dvc_more ON DATE:2019-07-03
ON ZONE: dvc_more ON DATE:2019-07-04
ON ZONE: dvc_more ON DATE:2019-07-05
ON ZONE: 54ft_road ON DATE:2019-06-28
ON ZONE: 54ft_road ON DATE:2019-06-30
ON ZONE: 54ft_road ON DATE:2019-07-01
ON ZONE: 54ft_road ON DATE:2019-07-02
ON ZONE: 54ft_road ON DATE:2019-07-03
ON ZONE: 54ft_road ON DATE:2019-07-04
ON ZONE: 54ft_road ON DATE:2019-07-05
ON ZONE: junction_mall ON DATE:2019-06-28
ON ZONE: junction_mall ON DATE:2019-06-30
ON ZONE: junction_mall ON DATE:2019-07-01
ON ZONE: junction_mall ON DATE:2019-07-02
ON ZONE: junction_mall ON DATE:2019-07-03
ON ZONE: junc

In [5]:
#NICE